In [2]:
import datetime

from langchain.chat_models import init_chat_model
from pywin.framework.toolmenu import tools
! pip install langgraph
! pip install langchain
! pip install langchain_community
! pip install python-dotenv

In [8]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from dotenv import load_dotenv

load_dotenv()

model = init_chat_model(
    model="deepseek-chat",
    base_url="https://api.deepseek.com",
    temperature=0.7,
    max_tokens=2048,
)

@tool
def get_current_date():
    """获取今天几号"""
    return datetime.datetime.today()

agent = create_agent(
    model=model,
    tools=[get_current_date],
    system_prompt="你是一个报时器",
)

response = agent.invoke({"message":[{"role": "user", "content": "今天几号"}]})
print(response)

{'messages': [AIMessage(content='我来帮您获取今天的日期。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 135, 'total_tokens': 150, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 7}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '0d2f4896-dc39-47bb-9d9a-952699b51d92', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--cf9c6793-c479-4a4c-8e11-5c7802dd34af-0', invalid_tool_calls=[{'type': 'invalid_tool_call', 'id': 'call_00_e6yBZ3SaB14Z0wdlL6jmwB8T', 'name': 'get_current_date', 'args': '', 'error': 'Function get_current_date arguments:\n\n\n\nare not valid JSON. Received JSONDecodeError Expecting value: line 1 column 1 (char 0)\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_

In [ ]:
def check_quality(state: DocWorkflowState) -> DocWorkflowState:
    """检查文档质量的节点（支撑选择边）"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", """你是严格的质量检查员，根据用户需求判断文档初稿是否合格。
        检查标准：1. 完全覆盖用户需求的核心要点；2. 逻辑清晰、无遗漏信息；3. 语言通顺、无语法错误。
        输出要求：
        - 质量检查结果：合格/不合格（仅二选一）
        - 检查反馈：详细说明合格的原因或不合格的改进点（1-2句话）"""),
        ("human", "用户需求：{user需求}\n文档初稿：{文档初稿}")
    ])

    chain = prompt | llm
    response = chain.invoke({
        "user需求": state.user需求,
        "文档初稿": state.文档初稿
    })

    # 解析 LLM 输出（简化解析，实际可用 LangChain 的 OutputParser）
    content = response.content.strip()
    检查结果 = "合格" if "合格" in content[:10] else "不合格"
    反馈 = content.split("\n")[-1] if "\n" in content else content

    # 更新状态：填充“质量检查结果”和“检查反馈”
    return DocWorkflowState(
        **state.dict(),
        质量检查结果=检查结果,
        检查反馈=反馈
    )